In [1]:
%load_ext google.datalab.kernel
%load_ext google.cloud.bigquery

from datetime import date
import os 
import subprocess
import time 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

today = date.today()
d1 = today.strftime("%d/%m/%Y")
d1

'01/07/2021'

In [3]:
def parse_vm_csv(csv_filename):
    
    df = pd.read_csv(csv_filename)
    df = df[['instance','zone']]
    df = df.sort_values('instance',ascending=True)
    df.to_csv('vm_parsed_data.csv')
    
    df_dict = df.to_dict()

    insatnces = list(df_dict['instance'].values())
    zones = list(df_dict['zone'].values())

    return df, (insatnces,zones) 


def start_vm(instance_name_str,zone_str):
    command = f'gcloud compute instances start --zone {zone_str}  {instance_name_str}'
    os.system(command)

def stop_vm(instance_name_str,zone_str):
    command = f'gcloud compute instances stop --zone {zone_str}  {instance_name_str}'
    os.system(command)
    
def ssh_login_and_run_commands_script(instance_name_str,zone_str ,project_name_str,script_name = r'./run_commands.sh'):
    script_commands = subprocess.call([ 'bash' , script_name ])
    #     script_commands = subprocess.call([ 'bash' , './' + script_name ])
    # command = f'gcloud beta compute ssh --zone {zone_str} {instance_name_str}  --project {project_name_str} --command {script_commands}'
    command = f'gcloud beta compute ssh --zone {zone_str} {instance_name_str}  --project {project_name_str} --command={script_commands}'
    os.system(command)

def run_one_vm_cycle(instance_name_str,zone_str ,project_name_str,script_name):

    start_vm(instance_name_str,zone_str)
    ssh_login_and_run_commands_script(instance_name_str,zone_str,project_name_str,script_name)
    stop_vm(instance_name_str,zone_str)


In [4]:
import pandas as pd

vm_metadata_csv_path = '/home/arpalus/Work_Eldad/Arpalus_Code/Eldad-Local/Eldad-GCP/Data/vm_metadata.csv'
df, _ = parse_vm_csv(vm_metadata_csv_path)

In [2]:
df = df.head(50)
df.to_csv('vm_parsed_data.csv')


NameError: name 'df' is not defined

In [6]:
# exp 

exp_df  = df.iloc[40:]
exp_df = exp_df.reset_index()
exp_df=exp_df.drop(['index'],axis=1)
instances,zones = exp_df['instance'].to_list(),exp_df['zone'].to_list()
vm_dict = {instance:zone for instance,zone in zip(instances,zones)}
vm_dict

{'instance-41': 'asia-east1-a',
 'instance-42': 'asia-northeast1-c',
 'instance-43': 'us-west4-a',
 'instance-44': 'us-east1-d',
 'instance-45': 'us-west1-a',
 'instance-46': 'us-east1-c',
 'instance-47': 'us-east4-b',
 'instance-48': 'us-east1-d',
 'instance-49': 'us-west1-b',
 'instance-50': 'us-east4-b'}

In [7]:
PROJECT_NAME_STR = 'shelfauditdec19'
SCRIPT_NAME = 'run_commands.sh'

run_one_vm_cycle(instance_name_str = 'instance-41',zone_str = 'asia-east1-a' ,project_name_str=PROJECT_NAME_STR,script_name=SCRIPT_NAME)
